In [ ]:
import config

config.set_environment()


In [ ]:
print(config.HUGGINGFACEHUB_API_TOKEN)

In [ ]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort="downloads", direction=-1)
):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular("text-classification")

In [ ]:
list_most_popular("summarization")

In [ ]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

In [ ]:
from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

In [ ]:
print(sentiment_model(customer_email))

In [ ]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

In [ ]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

In [ ]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


In [ ]:
import pandas as pd

# Specify the file path
file_path = "../data/Cases.csv"

# Read the CSV file into a dataframe
df = pd.read_csv(file_path)

# Display the dataframe
df.head()


In [ ]:
type_set = set(df['Type'].unique())

print(type_set)

In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
template_intro = "Given this text, decide what is the issue the customer is concerned about. Valid categories are these:\n"
template_categories = ""
for category in type_set:
    template_categories += f"* {category}\n"
template_outro = "Text: {email}\nCategory:"

template = template_intro + template_categories + template_outro

prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
df['Category'] = None
i = 0
for customer_email in df['Subject'] + ': ' + df['Description']:
    #print(customer_email)
    category = llm_chain.run(customer_email)
    i += 1
    print(f"{i}/{len(df)}")
    df['Category'][i] = category





In [36]:
print(f"{df.head()['Category']}/{df.head()['Type']}")

0                     None
1     OtherSoftwareProblem
2            Service Issue
3                      nan
4                      nan
Name: Category, dtype: object/0    Service Issue
1      Maintenance
2      Maintenance
3      Maintenance
4      Maintenance
Name: Type, dtype: object


In [35]:
df['Type_category_match'] = df['Type'] == df['Category']
df['Type_category_match'].value_counts()

Type_category_match
False    102595
Name: count, dtype: int64

In [37]:
df.to_csv('../output/Cases_with_intention_category.csv', index=False)


In [ ]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))

In [ ]:
import sys
from pathlib import Path
# Add the root folder to sys.path
import os

current_working_directory = %pwd
print(current_working_directory)
file_path = current_working_directory


root_path = Path(file_path).resolve().parents[1]  # Adjust the number of parents based on your structure
sys.path.append(str(root_path))

from config import set_environment
from config import set_environment

#from ..config import set_environment

set_environment()

In [ ]:
from langchain import HuggingFaceHub

# facebook model did not work maybe because there are more steps to get access, than just the API key
summarizer = HuggingFaceHub(
    repo_id="google-t5/t5-small",
    #repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)   

def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

In [ ]:
from transformers import pipeline

summarize_model = pipeline(
    task="summarization",
    model="philschmid/bart-large-cnn-samsum"
)

In [ ]:
print(summarize_model(customer_email))